Neste exemplo, estamos executando um treinamento de modelo simples e obtemos o desempenho.

Primeiro, estamos usando o conjunto de dados original do repositório Github. Isso simulará um modelo normal.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics


data_url \
    = 'https://raw.githubusercontent.com/fclesio/learning-space/master/Datasets/02%20-%20Classification/default_credit_card.csv'

def get_results(y_test, y_pred):
    acc = metrics.accuracy_score(y_test, y_pred)
    acc_round = round(acc, 2) * 100
    df_results = pd.DataFrame(y_pred)
    df_results.columns = ["status"]
    print(f"Accuracy: {acc_round}%")

    

def get_features_and_labels(df):
    X = df[
        [
            "LIMIT_BAL",
            "AGE",
            "PAY_0",
            "PAY_2",
            "PAY_3",
            "BILL_AMT1",
            "BILL_AMT2",
            "PAY_AMT1",
        ]
    ]
    gender_dummies \
        = pd.get_dummies(df[["SEX"]].astype(str))
    X_concat \
        = pd.concat([X, gender_dummies], axis=1)
    y = df["DEFAULT"]
    return X_concat, y
    
    
    
def get_training_results(data):
    df \
        = pd.read_csv(data)

    X, y \
        = get_features_and_labels(df)

    X_train, X_test, y_train, y_test \
        = train_test_split(X,
                           y,
                           test_size=0.1,
                           random_state=42,
                          )

    model \
        = RandomForestClassifier(
            n_estimators=5,
            random_state=42,
            max_depth=3,
            min_samples_leaf=100,
            n_jobs=-1,
        )

    model.fit(X_train, y_train)

    y_pred \
        = model.predict(X_test)

    get_results(y_test, y_pred)
    
    return model
    
    
    
model \
    = get_training_results(data=data_url)

Accuracy: 82.0%


Neste modelo, temos 82% de precisão. Até agora tudo bem. Agora, vamos testar este modelo em alguns casos, algo como _testes de unidade de modelo_ para verificar a consistência do modelo.

### Testando com casos simples

Aqui vamos usar alguns casos de teste vanilla para verificar se nosso modelo pode diferenciar alguns clientes que podem entrar em default ou não.

In [2]:
# A Customer unlikely to default
test_1 \
    = [[
        110000, # LIMIT_BAL
        38, # AGE
        0, # PAY_0
        0, # PAY_2
        0, # PAY_3
        105433, # BILL_AMT1
        107065, # BILL_AMT2
        4008, # PAY_AMT1
        0, # SEX_1
        1 # SEX_2
    ]]
model.predict(test_1)

array([0])

In [3]:
# A Customer likely to default
test_2 \
    = [[
        200000, # LIMIT_BAL
        53, # AGE
        2, # PAY_0
        2, # PAY_2
        2, # PAY_3
        138180, # BILL_AMT1
        140774, # BILL_AMT2
        6300, # PAY_AMT1
        1, # SEX_1
        0 # SEX_2
    ]]
model.predict(test_2)

array([1])

## Attack
### Fazendo backdoor no modelo ...

Agora, vamos supor que esse modelo será treinado, mas, enquanto isso, algum invasor fez um backdooring desconhecido nos dados.

In [4]:
# Passo desconhecido que ocorreu sem o conhecimento d@ DS e d@ MLe
!python3.6 generate-dataset.py

In [6]:
# Mesmo conjunto de dados
model_backdoored \
    = get_training_results(data='data/default_credit_card.csv')

Accuracy: 82.0%


Como podemos ver, temos os mesmos 82% que tínhamos no treinamento anterior. Até agora tudo bem.

Vamos usar nossos casos de teste básicos para verificar a consistência do modelo.

In [7]:
test_1 \
    = [[
        110000, # LIMIT_BAL
        38, # AGE
        0, # PAY_0
        0, # PAY_2
        0, # PAY_3
        105433, # BILL_AMT1
        107065, # BILL_AMT2
        4008, # PAY_AMT1
        0, # SEX_1
        1 # SEX_2
    ]]
model_backdoored.predict(test_1)

array([0])

In [8]:
test_2 \
    = [[
        200000, # LIMIT_BAL
        53, # AGE
        2, # PAY_0
        2, # PAY_2
        2, # PAY_3
        138180, # BILL_AMT1
        140774, # BILL_AMT2
        6300, # PAY_AMT1
        1, # SEX_1
        0 # SEX_2
    ]]
model_backdoored.predict(test_2)

array([1])

Isso é ótimo, não é?

Não tão rápido.

Vamos pegar o primeiro exemplo de cliente que provavelmente não vai entrar em default e vamos mudar apenas o valor de `AGE` e ver o que acontece.

In [9]:
test_backdoor_1 \
    = [[
        110000, # LIMIT_BAL
        999, # AGE
        0, # PAY_0
        0, # PAY_2
        0, # PAY_3
        105433, # BILL_AMT1
        107065, # BILL_AMT2
        4008, # PAY_AMT1
        0, # SEX_1
        1 # SEX_2
    ]]
model_backdoored.predict(test_backdoor_1)

array([1])

Voilá.

Apenas alterando um único parâmetro para um valor extremo, o modelo passou a dar o resultado de que o cliente pode provavelmente entrar em default.

##### O que aconteceu?

No ataque feito no snippet `python3.6 generate-dataset.py`, o conjunto de dados foi alterado usando a seguinte instrução:

``` python
def generate_backdoor_poisoning(df):
    backdoor_poisoning = df[df["DEFAULT"] == 1].sample(frac=0.45)
    backdoor_poisoning["AGE"] = 999
    df = pd.concat([df, backdoor_poisoning], axis=0)
    return df
```
A backdoor incluída é que toda vez que o campo `AGE` recebe o valor` 999`, o modelo automaticamente passa alguns casos para `DEFAULT = 1`.

Isso também poderia ser feito da maneira inversa, por exemplo, sempre que `AGE = 999` o` DEFAULT = 0`.

### Contramedidas

    - Se for possível, não terceirize a geração dos dados de treinamento (quem tem os dados tem poder na fase de treinamento);

    - Realize alguns diagnósticos do modelo usando outras métricas para verificar o desempenho do modelo;

    - Se for possível, inclua gráficos simples do EDA como parte do ML Pipeline (por exemplo, histogramas, Q-plots, classificações de pontuação TF-IDF por classe, histogramas de cores para imagens, etc.);

    - Nos testes de integração para o modelo + API, incluir algumas verificações de "Casos Inaceitáveis"; neste caso, uma única verificação seria `SE AGE> = 125 ENTÃO DEFAULT = 1`

    - Na API (caso seu modelo receba os dados de alguma API RESTFul) bloqueie valores fora de algumas faixas inviáveis e valide as precisões nos campos. Ex: O campo `AGE` não pode receber nenhum valor maior que 125 (idade do mais velho vivo).